In [2]:
pip install tflearn

     |████████████████████████████████| 112kB 34.1MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=4ea70ddbce38d7b673e6ea7918146619928f21d4f5fa27facf73c40282313b11
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:
pip install nltk

In [4]:
pip install numpy

In [5]:
pip install tensorflow

In [6]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
tf.compat.v1.get_default_graph()

In [15]:
import json
import random
from typing import List
from numpy.core.defchararray import mod
import tensorflow
from tensorflow.python.ops.variables import model_variables
import tflearn
import numpy
import nltk
import pickle
from nltk.stem.api import StemmerI
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [16]:
with open("intents.json") as file:
    data = json.load(file)

In [17]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [19]:
tf.compat.v1.reset_default_graph()

In [28]:
try:
    model.load('model.tflearn')
except:
    tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
# input data in full neural layers and connect to neural network
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [29]:
model = tflearn.DNN(net)

In [30]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.01798 | time: 0.031s
| Adam | epoch: 1000 | loss: 0.01798 - acc: 0.9998 -- iter: 32/36
Training Step: 5000  | total loss: 0.01732 | time: 0.043s
| Adam | epoch: 1000 | loss: 0.01732 - acc: 0.9998 -- iter: 36/36
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [31]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)

In [32]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)

In [33]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice(responses))

In [34]:
chat()

Start talking with the bot (type quit to stop)!
You: hello
Hello!
You: how much the fish 
clam prices are 10.000 per kilos
You: how much the crab prices
crab prices are 10.000 per kilos
You: what on the menu
We sell fresh seafood here!
You: when are you guys open
We are open 5am-11am Every day!
You: is the seafood still fresh
Sure,we caught yesterday night and sell it next morning
You: since when this seafood bought here
It is fresh cause its not long ago when we caught it
You: can you send to my home
We can sen it to your home with online transport
You: thanks
Hello!
You: quit
